In [2]:
# Loading dependencies
import pandas as pd


In [4]:
#Loading data
df = pd.read_csv("./data/skelbiu.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1009 non-null   object
 1   title       1009 non-null   object
 2   price       1009 non-null   object
 3   link        1008 non-null   object
 4   image       1007 non-null   object
 5   condition   1007 non-null   object
 6   specs       1007 non-null   object
dtypes: object(7)
memory usage: 55.3+ KB


In [5]:
#Cleaning data
df.dropna(inplace=True)

In [11]:
df.loc[df.price == "None"]

,Unnamed: 0,title,price,link,image,condition,specs
31,31,Nupirksiu sugedusia vaizdo plokšte Rtx Serijos.,None,https://www.skelbiu.lt/skelbimai/nupirksiu-sug...,https://skelbiu-img.dgn.lt/1_18_3518840364/nup...,"24,51 €",Nupirksiu sugedusia vaizdo plokšte RTX SERIJOS...
128,127,Nupirksiu Geforce Rtx 3090 ti vaizdo plokštę.,None,https://www.skelbiu.lt/skelbimai/nupirksiu-gef...,https://skelbiu-img.dgn.lt/1_18_3526055508/nup...,"24,51 €",None
130,129,Vaizdo plokštė asus Ex-gtx1050-2g 50e,None,https://www.skelbiu.lt/skelbimai/vaizdo-plokst...,https://skelbiu-img.dgn.lt/1_18_3522011880/vai...,Naudota,None
454,453,Nupirkčiau Rtx 2060 Super,None,https://www.skelbiu.lt/skelbimai/nupirkciau-rt...,https://skelbiu-img.dgn.lt/1_18_3075251721/rtx...,Naudota,"Nupirkčiau RTX 2060 Super, dar galima keisti į..."
463,462,Ieškau 1080 aplink klaipėda siulyti,None,https://www.skelbiu.lt/skelbimai/ieskau-1080-a...,https://skelbiu-img.dgn.lt/1_18_3518646312/ies...,Naudota,Sveiki ieskau 1080 arba 1080ti.su kainom ir pa...
485,484,Pirksiu Isa interfeiso seną video plokštę.,None,https://www.skelbiu.lt/skelbimai/pirksiu-isa-i...,https://skelbiu-img.dgn.lt/1_18_3353306318/pir...,Naudota,"Pirksiu seną video plokštę ISA interfeiso, vei..."
517,516,Sapphire R9 270x Oc 2gb,None,https://www.skelbiu.lt/skelbimai/sapphire-r9-2...,https://skelbiu-img.dgn.lt/1_18_3527413698/sap...,Naudota,GPU: Sapphire Dual-X R9 270X OC 2GB - AMD Rade...
534,533,Gtx Titan Xp,None,https://www.skelbiu.lt/skelbimai/gtx-titan-xp-...,https://skelbiu-img.dgn.lt/1_18_3510394926/tit...,"24,51 €",Ieskau Titan Xp pirkciau 1-3 vienetus
535,534,"Vaizdo plokstes, gpu",None,https://www.skelbiu.lt/skelbimai/vaizdo-plokst...,https://skelbiu-img.dgn.lt/1_18_3517090462/vai...,Naudota,gigabyte 1070ti PARDUOTA\nPALIT 2070 (300e)\nG...
592,591,"1050ti, 1660 super, Rx 580 8gb red devil.",None,https://www.skelbiu.lt/skelbimai/1050ti-1660-s...,https://skelbiu-img.dgn.lt/1_18_3482677976/105...,Naudota,Parduodamos 3 tvarkingos vaizdo plokstes.\n105...


In [13]:
df.drop(df.index[(df["price"] == "None")],axis=0,inplace=True)

In [15]:
df.loc[df.specs == "None"]

,Unnamed: 0,title,price,link,image,condition,specs
59,58,Vaizdo plokste,10 €,https://www.skelbiu.lt/skelbimai/vaizdo-plokst...,https://skelbiu-img.dgn.lt/1_18_703335425/vaiz...,Naudota,None
125,124,Ge Force Gt 7300 vaizdo plokštė,10 €,https://www.skelbiu.lt/skelbimai/ge-force-gt-7...,https://skelbiu-img.dgn.lt/1_18_3511060626/ge-...,Naudota,None
129,128,Nvidia Vaizdo Plokštė (Gpu) Zotac Gtx 680,50 €,https://www.skelbiu.lt/skelbimai/nvidia-vaizdo...,https://skelbiu-img.dgn.lt/1_18_3524029302/nvi...,Naudota,None
135,134,Vaizdo plokštė Gigabyte Radeon Rx 580 8 GB,150 €,https://www.skelbiu.lt/skelbimai/vaizdo-plokst...,https://skelbiu-img.dgn.lt/1_18_3534416400/vai...,Naudota,None
138,137,Vaizdo plokštė Geforce Rtx 3080 Suprim 10g non...,800 €,https://www.skelbiu.lt/skelbimai/vaizdo-plokst...,https://skelbiu-img.dgn.lt/1_18_3520039052/vai...,Naudota,None
439,438,Gpu 1gb geforce,20 €,https://www.skelbiu.lt/skelbimai/gpu-1gb-gefor...,https://skelbiu-img.dgn.lt/1_18_3508928662/gpu...,Naudota,None
482,481,Amd Radeon R9 290,80 €,https://www.skelbiu.lt/skelbimai/amd-radeon-r9...,https://skelbiu-img.dgn.lt/1_18_3525223744/amd...,Naudota,None
491,490,Ieškau Rx 6800 / 6800xt,400 €,https://www.skelbiu.lt/skelbimai/ieskau-rx-680...,https://skelbiu-img.dgn.lt/1_18_3472388423/ies...,Nauja,None
506,505,Asus Rog Strix 1080,300 €,https://www.skelbiu.lt/skelbimai/asus-rog-stri...,https://skelbiu-img.dgn.lt/1_18_3528517226/asu...,Naudota,None
526,525,Perku 6800xt,400 €,https://www.skelbiu.lt/skelbimai/perku-6800xt-...,None,Naudota,None


In [16]:
df.drop(df.index[(df["specs"] == "None")],axis=0,inplace=True)

In [28]:
df.drop(df.index[(df["condition"] != 'Naudota')],axis=0,inplace=True)

In [29]:
df.groupby('condition').count()

,Unnamed: 0,title,price,link,image,specs
condition,,,,,,
Naudota,863,863,863,863,863,863


In [31]:
df['price_int'] = df.price.apply(lambda x: x.split()[0])


In [38]:
df.drop(df.index[(df["price_int"] == "13,90")],axis=0,inplace=True)

In [39]:
df['price_int'] = df['price_int'].astype(float)

In [40]:
df.drop(df[(df['price_int'] < 15)].index, inplace=True)

In [42]:
df

,Unnamed: 0,title,price,link,image,condition,specs,price_int
1,1,1060 6gb msi armor oc,120 €,https://www.skelbiu.lt/skelbimai/1060-6gb-msi-...,https://skelbiu-img.dgn.lt/1_18_3538924964/106...,Naudota,Viewsonic XG2405-2 144hz 24 coliu VA 200 eur\n...,120.0
2,2,Rtx 3070 Asus Dual oc Lietuviska garantija iki...,400 €,https://www.skelbiu.lt/skelbimai/rtx-3070-asus...,https://skelbiu-img.dgn.lt/1_18_3538854628/rtx...,Naudota,"Sveiki , parduodu tvarkinga RTX 3070\nasus dua...",400.0
3,3,Vaizdo ploksciu supirkimas aukstomis kainomis,30 000 €,https://www.skelbiu.lt/skelbimai/vaizdo-ploksc...,https://skelbiu-img.dgn.lt/1_18_3464085469/vai...,Naudota,"Sveiki, del eth zlugimo atsirado daug zmoniu i...",30.0
4,4,Gigabyte Aorus Geforce Rtx 2060 6gb,259 €,https://www.skelbiu.lt/skelbimai/gigabyte-aoru...,https://skelbiu-img.dgn.lt/1_18_3538579528/gig...,Naudota,Parduodu naudota Gigabyte AORUS GeForce RTX 20...,259.0
6,6,Parduodu Rtx 3060ti Tuf-rtx3060ti-8g-gaming,400 €,https://www.skelbiu.lt/skelbimai/parduodu-rtx-...,https://skelbiu-img.dgn.lt/1_18_3539273380/par...,Naudota,Apie metu gal tik naudota nes visa laika tik r...,400.0
...,...,...,...,...,...,...,...,...
1002,1001,Msi Geforce Rtx 2060 Super Gaming X 8gb su Rgb,260 €,https://www.skelbiu.lt/skelbimai/msi-geforce-r...,https://skelbiu-img.dgn.lt/1_18_3536639620/msi...,Naudota,MSI GeForce RTX 2060 SUPER GAMING X 8GB su RGB...,260.0
1004,1003,Msi Geforce Rtx 3070 Suprim X,700 €,https://www.skelbiu.lt/skelbimai/msi-geforce-r...,https://skelbiu-img.dgn.lt/1_18_3523097418/msi...,Naudota,"Nemaininta, neardyta, be defektu. Yra dezute i...",700.0
1006,1005,Parduodama Zotac Geforce Gtx770 2gb vaizdo korta,45 €,https://www.skelbiu.lt/skelbimai/parduodama-zo...,https://skelbiu-img.dgn.lt/1_18_3496244750/par...,Naudota,Parduodu Zotac GeForce GTX 770 2GB vaizdo blok...,45.0
1007,1006,Gigabyte Gtx 1080 Turbo Oc,200 €,https://www.skelbiu.lt/skelbimai/gigabyte-gtx-...,https://skelbiu-img.dgn.lt/1_18_3508494544/gig...,Naudota,Pakeista pasta į Thermal Grizzly Kryonout. Vei...,200.0
